# Tubes a choc

In [ ]:
from trustutils import run

run.introduction('Y. GORSSE, M. NDJINGA')

### Description 
Issus du chapitre 10 du livre de Eleuterio F. Toro, Riemann Solvers and Numerical Methods for Fluid Dynamics, Third edition, Springer, 2009. 

Couramment utilisés comme benchmark pour le comportement de        méthodes numériques dans les ouvrages et publications scientifiques, ainsi que pour la vérification du comportement de certains codes.

Objectifs : montrer la la robutesse du code et évaluer 

1. l'apparition d'oscillations parasites, 
2. la propagation des ondes avec la bonne vitesse, 
3. la capture des états intermédiaires à la bonne hauteur.

In [ ]:
run.TRUST_parameters()

In [ ]:
from trustutils import run, plot
import os

run.reset()
run.initBuildDirectory()

origin, build_dir = os.getcwd(), run.BUILD_DIRECTORY
os.chdir(build_dir)
exec(open("main.py").read())
os.chdir(origin)

list_meshes = [100, 800]
list_tests = ['Toro1','Toro2','Toro4','Toro5','Toro6','Toro7','PWR1','PWR2','PWR3']

for m in list_meshes:
    for dis in ["PolyMAC_P0", "VDF"]:
        for test in list_tests:
            run.addCase(f"n{m}/{dis}", f"{test}.data")

run.printCases()
run.runCases()

## Description des cas tests

These tests are designed to assess the robustness and accuracy of numerical methods at the core of the solver, independently from the boundary condition, and source term treatment involving correla       tions.

They consist in the numerical resolution of the Shock tube problem for a perfect gas with $\gamma=1.4$. 

The initial state consists in two constant states $W_L=(\rho_L,u_L,p_L)$ and $W_R=(\rho_R,u_R,p_R)$ separated by a discontinuity at $x=x_d$. The following table gives the values of $W_L$ and $W_R$ for each test.

In [ ]:
display(tab)

In [ ]:
import time
from trustutils import plot

variables = ["RHO", "P", "V", "EINT"]

dis_m = {"VDF" : "x-", "PolyMAC_P0" : "-"}

for test in list_tests:
    columns = plot.readFile(f"{test}.ex")
    var = columns[0].split()
    data = plot.loadText(f"{test}.ex", skiprows=1)
    
    a = plot.Graph(nX=2, nY=2, title=test)
    nX, nY = 0, 0
    for p in variables:
        a.addPlot([nX, nY], f"Profil de {p}")
        for m in list_meshes:
            for dis, ma in dis_m.items():
                a.addSegment(f"n{m}/{dis}/{test}_{p}.son", label=f"{dis} n={m}",lw = 4, marker=ma)
        nb = var.index(p)
        a.add(data[0], data[nb], label="Exact solution", lw=2, color='k')
        a.label("z [m]", p)
        
        nX = nX+1 if nY%2 else nX
        nY = (nY+1)%2